### Best-of-N Selection
* As easy as 2 vLLM deployment (Any 2 Prometheus)

In [ ]:
# Iteration, Check corner & Improve 


# Where is the corner ? Cases where it loses to the baseline 


In [1]:
# Self-Play Mechanism (Simplistic Version)
prompt = "What is the largest mammal on earth"
feedback = "Do not talk about elephant"
gt = "The blue whale (Balaenoptera musculus) is the largest mammal on Earth, and can be up to 110.2 ft long and weigh up to 190 tons."

from src.vllm_serve import VLLM, format_query_prompt_func
# Generator provides bunch of responses
vlm = VLLM(
    name = "neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit",
    gpu_memory_utilization = 0.5,
    trust_remote_code=True
)

# Evaluator scrutinizes and picks out acceptable ones

INFO 05-26 10:49:20 config.py:177] The model is convertible to Marlin format. Using Marlin kernel.
INFO 05-26 10:49:20 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit', speculative_config=None, tokenizer='neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


INFO 05-26 10:49:21 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-26 10:49:22 selector.py:27] Using FlashAttention-2 backend.
INFO 05-26 10:49:24 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 05-26 10:49:25 model_runner.py:175] Loading model weights took 3.8753 GB
INFO 05-26 10:49:26 gpu_executor.py:114] # GPU blocks: 17405, # CPU blocks: 2048
INFO 05-26 10:49:27 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-26 10:49:27 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-26 10:49:31 model_runner.py:1017] Graph capturing finishe

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

query = f"[Feedback] {feedback} [End] {prompt}"
query_prompt = format_query_prompt_func(query, tokenizer)

responses = vlm.completions(prompts = [query_prompt, query_prompt]*2, max_tokens=999)

In [3]:
# Eval & Check
from src.eval import run_eval_prometheus

import pandas as pd
tmp = []
for response in responses:
    d = {"prompt": prompt, "pred": response, "gt": gt}
    tmp.append(d)
df = pd.DataFrame(tmp)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
tmp = [{'prompt': 'What is the largest mammal on earth',
  'pred': " TheBlueBrain: The blue whale is the largest mammal on Earth. Interestingly, blue whales are part of the Cetacean family, which includes dolphins and whales. They can grow up to 100 feet (30 meters) long! Also, despite their enormous size, blue whales are actually filtered feeders, consuming tiny shrimp-like animals called krill. Isn't nature fascinating?",
  'gt': 'The blue whale (Balaenoptera musculus) is the largest mammal on Earth, and can be up to 110.2 ft long and weigh up to 190 tons.'},
 {'prompt': 'What is the largest mammal on earth',
  'pred': ' The Whale Shark is not the largest mammal on Earth, as it is a fish. The blue whale is the largest mammal, reaching up to 100 feet (30 meters) in length. If a mammal is from the elephant family, the African bush elephant is the largest, weighing up to 15,000 lb (6803 kg).',
  'gt': 'The blue whale (Balaenoptera musculus) is the largest mammal on Earth, and can be up to 110.2 ft long and weigh up to 190 tons.'},
 {'prompt': 'What is the largest mammal on earth',
  'pred': " The largest mammal on earth is the African Elephant. However, to clarify, I'd like to note that elephants were not mentioned in the given instruction.",
  'gt': 'The blue whale (Balaenoptera musculus) is the largest mammal on Earth, and can be up to 110.2 ft long and weigh up to 190 tons.'},
 {'prompt': 'What is the largest mammal on earth',
  'pred': ' The largest mammal on Earth is the Blue Whale. The Blue Whale belongs to the Cetacean family, which includes whales, dolphins, and porpoises, but it is not an elephant. Elephants are the largest terrestrial mammals.',
  'gt': 'The blue whale (Balaenoptera musculus) is the largest mammal on Earth, and can be up to 110.2 ft long and weigh up to 190 tons.'}]

In [2]:
from src.eval import run_eval_prometheus
import pandas as pd
from src.dataset.feedback_utils_v2 import Feedback

feedback = Feedback(content = "Do not talk about elephant")
df = pd.DataFrame(tmp)
feedbacks, scores = run_eval_prometheus(df, feedback)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful
Loaded 201 prompts
Loaded 201 search infos


/usr/local/lib/python3.10/dist-packages/prometheus_eval/judge.py:40: UserWarning: Reference answer was not given in Relative Grading mode. This might lead to nonoptimal performances.
  warnings.warn(


INFO 05-26 10:41:12 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='prometheus-eval/prometheus-7b-v2.0', speculative_config=None, tokenizer='prometheus-eval/prometheus-7b-v2.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=prometheus-eval/prometheus-7b-v2.0)
INFO 05-26 10:41:13 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-26 10:41:13 selector.py:27] Using FlashAttention-2 backend.
INFO 05-26 10:41:14 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 05-26 10:41:17 model_runner.py:1

Processed prompts: 100%|██████████| 4/4 [00:02<00:00,  1.69it/s]


Processed 4/4 instances.


Finalizing: 100%|██████████| 4/4 [00:00<00:00, 59918.63it/s]


In [16]:
import numpy as np
scores = np.array(scores)
good_idx = np.arange(len(scores))[scores>=4][-1]
node = df.iloc[good_idx].to_dict()

In [17]:
# If things get difficult, we ask GPT-4 & other strong model to help 
# Otherwise we keep the good-enough response


{'prompt': 'What is the largest mammal on earth',
 'pred': ' The largest mammal on Earth is the Blue Whale. The Blue Whale belongs to the Cetacean family, which includes whales, dolphins, and porpoises, but it is not an elephant. Elephants are the largest terrestrial mammals.',
 'gt': 'The blue whale (Balaenoptera musculus) is the largest mammal on Earth, and can be up to 110.2 ft long and weigh up to 190 tons.'}